# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use dataframes to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!

In [23]:
from collections import Counter
import json
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import string

## 1. Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
...,...,...,...
183526,Baby Teething Necklace for Mom Pretty Donut Sh...,Such a great idea! very handy to have and look...,5
183527,Baby Teething Necklace for Mom Pretty Donut Sh...,This product rocks! It is a great blend of fu...,5
183528,Abstract 2 PK Baby / Toddler Training Cup (Pink),This item looks great and cool for my kids.......,5
183529,"Baby Food Freezer Tray - Bacteria Resistant, B...",I am extremely happy with this product. I have...,5


### Build the word count vector for each review

Let us explore a specific example of a baby product.

In [4]:
products.iloc[269, :]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuation for the sake of simplicity. A smarter approach would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuation.

In [5]:
# fill missing values in the review column
products = products.fillna({'review':''})

def remove_punctuation(text):
    translator = text.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text

products['review_clean'] = products['review'].apply(lambda x: remove_punctuation(x) if x != '' else '')

In [6]:
products['word_count'] = products.review_clean.apply(lambda x: Counter(word.lower() for word in x.split())
                                                        if x != '' else {})
products.head()

,name,review,rating,review_clean,word_count
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...,"{'these': 1, 'flannel': 1, 'wipes': 3, 'are': ..."
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa..."
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'..."
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we..."
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'..."


Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [7]:
products.loc[269, 'word_count']

Counter({'a': 1, 'favorite': 1, 'in': 1, 'our': 1, 'house': 1})

### Extract sentiment

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
products = products[products.rating != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
products['sentiment'] = products.rating.apply(lambda rating: +1 if rating > 3 else -1)
products.head()

,name,review,rating,review_clean,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{'it': 3, 'came': 1, 'early': 1, 'and': 3, 'wa...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{'very': 1, 'soft': 1, 'and': 2, 'comfortable'...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{'this': 4, 'is': 4, 'a': 2, 'product': 2, 'we...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{'all': 2, 'of': 1, 'my': 1, 'kids': 2, 'have'...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{'when': 2, 'the': 6, 'binky': 3, 'fairy': 3, ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

### Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [10]:
# train_data, test_data = products.random_split(0.8, seed=1)
# print(len(train_data))
# print(len(test_data))

In order to get the correct results for the quiz, you need to read in a list of train and test indices from a set of json files instead.

In [11]:
with open('W1-assignment-train-idx.json', 'r') as f:
    test_indices = json.load(f)
train_data = products.iloc[test_indices]

with open('W1-assignment-test-idx.json', 'r') as f:
    test_indices = json.load(f)
test_data = products.iloc[test_indices]

print(len(train_data))
print(len(test_data))

133416
33336


### Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. General steps for extracting word count vectors are as follows:
- Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
- Compute the occurrences of the words in each review and collect them into a row vector.
- Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
- Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

Keep in mind that the test data must be transformed in the same way as the training data.

In [12]:
# use this token pattern to keep single-letter words
vectoriser = CountVectorizer(token_pattern=r'\b\w+\b')
# first, learn vocabulary from the training data and assign columns to words
# then convert the training data into a sparse matrix
train_matrix = vectoriser.fit_transform(train_data.review_clean)
# second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectoriser.transform(test_data.review_clean)

## 2. Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

In [13]:
sentiment_model = LogisticRegression(solver='liblinear')
sentiment_model.fit(train_matrix, train_data.sentiment)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

There are a total of `121713` coefficients in the model (`121712` plus the intercept). Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

Now that we have fitted the model, we can extract the weights (coefficients):

In [14]:
weights = sentiment_model.coef_
print(weights)
print(weights.shape)

[[-1.23995580e+00  2.73385478e-05  2.75426081e-02 ...  1.26133766e-02
   2.70133082e-03 -4.12496602e-05]]
(1, 121712)


In [15]:
num_positive_weights = np.count_nonzero(weights >= 0)
num_negative_weights = np.count_nonzero(weights < 0)

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 87104 
Number of negative weights: 34608 


### Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [16]:
sample_test_data = test_data.iloc[10:13, :]
sample_test_data

,name,review,rating,review_clean,word_count,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,"{'absolutely': 1, 'love': 1, 'it': 2, 'and': 2...",1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,"{'would': 2, 'not': 2, 'purchase': 1, 'again':...",-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,"{'was': 2, 'so': 1, 'excited': 1, 'to': 3, 'ge...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [17]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [18]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  For each row, the **score** (or margin) is a number in the range **(-inf, inf)**. Use a pre-built function to calculate the score of each data point in **sample_test_data**. In scikit-learn, you can call the decision_function() function.

**Hint**: You probably need to convert sample_test_data into the sparse matrix format first.

In [19]:
sample_test_matrix = vectoriser.transform(sample_test_data.review_clean)
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.60159415  -3.16903527 -10.42328687]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y_i} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions for sample_test_data:

In [20]:
preds = [1 if score > 0 else -1 for score in scores]
print(preds)

[1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from Turi Create.

In [21]:
print("Class predictions according to sklearn:")
print(sentiment_model.predict(sample_test_matrix))

Class predictions according to sklearn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the ones obtained from **sentiment_model**. The logistic regression classifier in scikit-learn comes with the **predict** function for this purpose.

### Probability predictions

We can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [25]:
probs = [1/(1+math.exp(-score)) for score in scores]
probs

[0.9963216070625329, 0.04034775288486938, 2.973110961576348e-05]

**Checkpoint**: Make sure your probability predictions match the ones obtained from Turi Create.

In [26]:
print("Class probabilities according to sklearn:")
print(sentiment_model.predict_proba(sample_test_matrix)[:, 1])

Class probabilities according to sklearn:
[9.96321607e-01 4.03477529e-02 2.97311096e-05]


**Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review? The third one.

## 3. Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`.
2.  Sort the data according to those predictions and pick the top 20.

In [27]:
probs = sentiment_model.predict_proba(test_matrix)[:, 1]
top_positive_idx = np.argsort(-probs)[:20]
most_positive = test_data.iloc[top_positive_idx]
most_positive

,name,review,rating,review_clean,word_count,sentiment
100166,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,"{'i': 28, 'bought': 1, 'this': 6, 'carrier': 7...",1
66059,"Evenflo 6 Pack Classic Glass Bottle, 4-Ounce",It's always fun to write a review on those pro...,5,Its always fun to write a review on those prod...,"{'its': 2, 'always': 3, 'fun': 1, 'to': 23, 'w...",1
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,"{'i': 38, 'absolutely': 1, 'love': 1, 'this': ...",1
22586,"Britax Decathlon Convertible Car Seat, Tiffany",I researched a few different seats to put in o...,4,I researched a few different seats to put in o...,"{'i': 26, 'researched': 2, 'a': 22, 'few': 3, ...",1
140816,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,"{'i': 23, 'bought': 1, 'this': 11, 'seat': 22,...",1
114796,"Fisher-Price Cradle 'N Swing, My Little Snuga...",My husband and I cannot state enough how much ...,5,My husband and I cannot state enough how much ...,"{'my': 1, 'husband': 1, 'and': 19, 'i': 3, 'ca...",1
137034,Graco Pack 'n Play Element Playard - Flint,My husband and I assembled this Pack n' Play l...,4,My husband and I assembled this Pack n Play la...,"{'my': 2, 'husband': 2, 'and': 13, 'i': 8, 'as...",1
168697,Graco FastAction Fold Jogger Click Connect Str...,Graco's FastAction Jogging Stroller definitely...,5,Gracos FastAction Jogging Stroller definitely ...,"{'gracos': 1, 'fastaction': 2, 'jogging': 2, '...",1
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,"{'great': 4, 'pram': 10, 'roccoi': 1, 'bought'...",1
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,"{'we': 3, 'are': 6, 'big': 2, 'best': 13, 'bot...",1


In [28]:
positive_products = np.unique(most_positive.name)
positive_products

array(['Baby Einstein Around The World Discovery Center',
       'Baby Jogger City Mini GT Double Stroller, Shadow/Orange',
       'Baby Jogger City Mini GT Single Stroller, Shadow/Orange',
       'Britax 2012 B-Agile Stroller, Red',
       'Britax Decathlon Convertible Car Seat, Tiffany',
       'Buttons Cloth Diaper Cover - One Size - 8 Color Options',
       'Diono RadianRXT Convertible Car Seat, Plum',
       'Evenflo 6 Pack Classic Glass Bottle, 4-Ounce',
       'Evenflo X Sport Plus Convenience Stroller - Christina',
       "Fisher-Price Cradle 'N Swing,  My Little Snugabunny",
       'Freemie Hands-Free Concealable Breast Pump Collection System',
       'Graco FastAction Fold Jogger Click Connect Stroller, Grapeade',
       "Graco Pack 'n Play Element Playard - Flint",
       'Ikea 36 Pcs Kalas Kids Plastic BPA Free Flatware, Bowl, Plate, Tumbler Set, Colorful',
       'Infantino Wrap and Tie Baby Carrier, Black Blueberries',
       'Mamas &amp; Papas 2014 Urbo2 Stroller - Black

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

In [29]:
top_negative_idx = np.argsort(probs)[:20]
most_negative = test_data.iloc[top_negative_idx]
most_negative

,name,review,rating,review_clean,word_count,sentiment
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,"{'we': 3, 'have': 3, 'not': 5, 'had': 1, 'any'...",-1
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,"{'this': 9, 'is': 7, 'the': 21, 'first': 3, 'r...",-1
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,"{'i': 18, 'thought': 1, 'it': 8, 'sounded': 1,...",-1
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,"{'i': 26, 'will': 4, 'try': 2, 'to': 29, 'writ...",-1
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,"{'this': 12, 'is': 15, 'my': 6, 'second': 3, '...",-1
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,"{'note': 1, 'we': 3, 'never': 1, 'installed': ...",-1
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,"{'we': 1, 'bought': 1, 'this': 5, 'baby': 8, '...",-1
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,"{'i': 12, 'bought': 1, 'this': 3, 'sprayer': 2...",-1
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,"{'its': 1, '3am': 3, 'in': 2, 'the': 11, 'morn...",-1
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,"{'do': 1, 'not': 2, 'buy': 1, 'this': 6, 'baby...",-1


In [30]:
negative_products = np.unique(most_negative.name)
negative_products

array(['Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months)',
       'Belkin WeMo Wi-Fi Baby Monitor for Apple iPhone, iPad, and iPod Touch (Firmware Update)',
       'Chicco Cortina KeyFit 30 Travel System in Adventure',
       'Cloth Diaper Sprayer--styles may vary',
       'Cosco Alpha Omega Elite Convertible Car Seat',
       'Ellaroo Mei Tai Baby Carrier - Hershey',
       'Fisher-Price Ocean Wonders Aquarium Bouncer',
       "Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)",
       'Motorola Digital Video Baby Monitor with Room Temperature Thermometer',
       'NUK Cook-n-Blend Baby Food Maker',
       'Peg-Perego Tatamia High Chair, White Latte',
       'Philips AVENT Newborn Starter Set',
       'Regalo My Cot Portable Bed, Royal Blue',
       'Safety 1st Deluxe 4-in-1 Bath Station',
       'Safety 1st Exchangeable Tip 3 in 1 Thermometer',
       'Safety 1st High-Def Digital Monitor',
       

### Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [40]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    preds = model.predict(data)
    # Compute the number of correctly classified examples
    num_correct = np.sum(true_labels == preds)
    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = num_correct / len(true_labels)
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [41]:
get_classification_accuracy(sentiment_model, test_matrix, test_data.sentiment)

0.9322654187664987

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76). 0.93

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better? No.

## 4. Train another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [42]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 'well', 'able', 'car', 'broke', 'less', 
                     'even', 'waste', 'disappointed', 'work', 'product', 'money', 'would', 'return']

In [43]:
len(significant_words)

20

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [47]:
vectoriser_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectoriser_word_subset.fit_transform(train_data.review_clean)
test_matrix_word_subset = vectoriser_word_subset.transform(test_data.review_clean)

### Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [53]:
simple_model = LogisticRegression(solver='liblinear')
simple_model.fit(train_matrix_word_subset, train_data.sentiment)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [54]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data.sentiment)

0.8693604511639069

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [60]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                        'coefficient': simple_model.coef_.flatten()})

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [65]:
simple_model_coef_table.sort_values(by='coefficient', ascending=False, inplace=True)
simple_model_coef_table

,word,coefficient
6,loves,1.673074
5,perfect,1.509812
0,love,1.363690
2,easy,1.192538
1,great,0.944000
4,little,0.520186
7,well,0.503760
8,able,0.190909
3,old,0.085513
9,car,0.058855


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [68]:
np.sum(simple_model_coef_table.coefficient > 0)

10

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**? Most, but not all.

In [95]:
model_coef_table = pd.DataFrame({'word': list(vectoriser.vocabulary_.keys()),
                                 'coefficient': sentiment_model.coef_.flatten()})

sentiment_model_positive_words = list(model_coef_table.loc[model_coef_table.coefficient > 0, 'word'])
positive_significant_words = list(simple_model_coef_table.loc[simple_model_coef_table.coefficient > 0, 'word'])

len(set(positive_significant_words).intersection(sentiment_model_positive_words))

7

## 5. Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [96]:
get_classification_accuracy(sentiment_model, train_matrix, train_data.sentiment)

0.9684745457816154

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [97]:
get_classification_accuracy(simple_model, train_matrix_word_subset, train_data.sentiment)

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set? The full model.

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [98]:
get_classification_accuracy(sentiment_model, test_matrix, test_data.sentiment)

0.9322654187664987

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [99]:
get_classification_accuracy(simple_model, test_matrix_word_subset, test_data.sentiment)

0.8693604511639069

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set? The full model again.

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**? The positive class.

In [100]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [102]:
num_positive  = (test_data['sentiment'] == +1).sum()
accuracy = num_positive / test_data.shape[0]
accuracy

0.8427825773938085

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)? Yes.